<a href="https://colab.research.google.com/github/AUT-Student/BigData-Project/blob/main/NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library

In [ ]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 30.0 MB/s 
     |████████████████████████████████| 101 kB 13.4 MB/s 
     |████████████████████████████████| 212 kB 63.1 MB/s 
     |████████████████████████████████| 1.1 MB 71.6 MB/s 
     |████████████████████████████████| 140 kB 68.7 MB/s 
     |████████████████████████████████| 596 kB 65.4 MB/s 
     |████████████████████████████████| 127 kB 64.9 MB/s 
     |████████████████████████████████| 94 kB 798 kB/s 
     |████████████████████████████████| 144 kB 56.9 MB/s 
     |████████████████████████████████| 271 kB 20.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's de

In [1]:
# General Libraries
import pandas as pd
import numpy as np
import requests
from copy import deepcopy

In [20]:
# Hugging Face Libraries
from transformers import pipeline, AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup, DataCollatorWithPadding

import datasets
from datasets import load_dataset, load_metric, Dataset

In [ ]:
# TF Libraries
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf

# Dataset

In [7]:
!gdown https://data.deepai.org/quora_question_pairs.zip

Downloading...
From: https://data.deepai.org/quora_question_pairs.zip
To: /content/quora_question_pairs.zip
100% 324M/324M [00:23<00:00, 14.0MB/s]


In [5]:
!unzip /content/quora_question_pairs.zip

Archive:  /content/quora_question_pairs.zip
replace sample_submission.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [6]:
!unzip /content/train.csv.zip

Archive:  /content/train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [8]:
dataset = pd.read_csv("/content/train.csv")

In [9]:
dataset

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,404287,537928,537929,What is one coin?,What's this coin?,0
404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


# Description



*   XLM RoBERTa (pretrainint) + Fine-Tuning (English) + Fine-Tuning (Persian)
*   Results on new articles
*   Results Cross Lingual
*   90% train, 10% test
*   10k translate with google translate



# Google Translate API

Reference: https://stackoverflow.com/a/48159904/8961642

In [15]:
def translate(text, url_id):
  target_language = "fa"
  source_language = "en"

  params = {"q": text, "target": target_language, "source": source_language}

  urls = [
          # armazochi@gmail.com        
          "https://script.google.com/macros/s/AKfycbxGEqdlejcKq59513TY54fdE_WKTxcdEg6spgeJB2AWAb-qziRsDiyfl_tRN7WBzGs3/exec",
          
          # armazochi1378@gmail.com
          "https://script.google.com/macros/s/AKfycbwC56gB1-5o3hQS8XUcuj6SfnX8uJZoiuyqCvUhYy9x6Sq2tH-vJ6vf6fWlL_1gJoE5uA/exec",

          # armazochi1@gmail.com
          "https://script.google.com/macros/s/AKfycbwoGoj0C8gYPGS_cacAVD92GS_CYzjpRvToIsNg-cxXC89dWoyIe9_oiyX7piXpW6ghrw/exec",

          # armazochi2@gmail.com
          "https://script.google.com/macros/s/AKfycbzSVXVHBgh7gaUtlh4zJNwBL5VcpZnRpoP14zARknUUkkcyGln2fIBe1sKrpFrUtffs/exec",

          # armazochi3@gmail.com
          "https://script.google.com/macros/s/AKfycbwvQnG-GCubuNmI5tbdCPzir9YNAx8Ya5xacBARANVT90k8ak53kYty_tupSIE2ll8/exec"

  ]

  url = urls[url_id]

  response = requests.get(url, params)

  return response.text

In [16]:
translate("Hello World! Hello Google Translate", 4)

'سلام دنیا! سلام گوگل ترنسلیت'

# Persian Dataset Creation

In [17]:
question_id_list = set(np.concatenate((dataset["qid1"].values, dataset["qid2"].values)))

In [18]:
len(question_id_list)

537933

In [19]:
question_list = [None] * (len(question_id_list)+1)

In [20]:
from numpy.ma.core import not_equal
for _, data in dataset.iterrows():
  qid1 = data["qid1"]
  question1 = data["question1"]

  qid2 = data["qid2"]
  question2 = data["question2"]

  if question_list[qid1] is None:
    question_list[qid1] = question1

  if question_list[qid2] is None:
    question_list[qid2] = question2

In [21]:
persian_question_list = [None] * (len(question_id_list)+1)

In [22]:
persian_question_list_copy = deepcopy(persian_question_list)

In [ ]:
for j in range(5):
  for i in range(92001, min(92001+(j+1)*4500, 112001)):
    if (persian_question_list[i] is None) or "Service invoked too many times for one day: translate" in persian_question_list[i]:
      if i%2000==0: print(f"i = {i}, j = {j}")
      persian_question_list[i] = translate(question_list[i], j)  

i = 94000, j = 0


In [ ]:
for i in range(72001, 92001):
  if (persian_question_list[i] is None) or "Service invoked too many times for one day: translate" in persian_question_list[i]:
    persian_question_list[i] = translate(question_list[i], 4)

In [ ]:
counter = 0
for i in range(72001, 92001):
  if "Service invoked too many times for one day: translate" in persian_question_list[i]:
    print(i)
    # persian_question_list[i] = translate(question_list[i])

In [ ]:
with open(r"/content/persian_question_list92k_index.txt", 'w') as fp:
    for i, item in enumerate(persian_question_list[:92001]):
        # write each item on a new line
        fp.write("%s\n" % str(i))
        fp.write("%s\n" % item)
    print('Done')

Done


In [ ]:
with open(r"/content/persian_question_list92k.txt", 'w') as fp:
    for item in persian_question_list[:92001]:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


# XLM RoBERTa

In [10]:
loss = tf.keras.losses.MeanSquaredError()

model = TFAutoModelForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path="xlm-roberta-base", num_labels=1, from_pt=True
)

All PyTorch model weights were used when initializing TFXLMRobertaForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Quora Dataset

In [11]:
quora_train_dataset = dataset[:40000]
quora_valid_dataset = dataset[40000:45000]
quora_test_dataset = dataset[45000:50000]

In [12]:
dataset = datasets.DatasetDict({"train": Dataset.from_pandas(quora_train_dataset),
                                "test": Dataset.from_pandas(quora_test_dataset),
                                "validation": Dataset.from_pandas(quora_valid_dataset)})

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate'],
        num_rows: 40000
    })
    test: Dataset({
        features: ['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate'],
        num_rows: 5000
    })
})

# Tokenizer

In [14]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

In [15]:
def preprocess_function(examples):
    return tokenizer(examples["question1"], examples["question2"], truncation=True)

In [16]:
preprocess_function(dataset["train"][:5])

{'input_ids': [[0, 4865, 83, 70, 29954, 390, 29954, 17997, 47, 14145, 23, 12008, 16839, 23, 68360, 32, 2, 2, 4865, 83, 70, 29954, 390, 29954, 17997, 47, 14145, 23, 12008, 16839, 32, 2], [0, 4865, 83, 70, 13765, 111, 29533, 73, 16495, 15, 12110, 127, 9, 14, 9, 5021, 748, 16, 69124, 32, 2, 2, 4865, 2806, 53383, 2174, 70, 42878, 27759, 89234, 70, 29533, 73, 16495, 15, 12110, 127, 9, 14, 9, 5021, 748, 16, 879, 15882, 4420, 32, 2], [0, 11249, 831, 87, 51312, 70, 38352, 111, 759, 1820, 94878, 12960, 17368, 10, 106137, 32, 2, 2, 11249, 831, 2665, 38352, 186, 124735, 390, 256, 41324, 8305, 100976, 32, 2], [0, 44084, 444, 87, 13893, 538, 4552, 459, 86, 538, 32, 11249, 831, 87, 86869, 442, 32, 2, 2, 26040, 70, 47143, 820, 3229, 378, 125458, 268, 3742, 8353, 24854, 2357, 8152, 34683, 125458, 268, 83, 101637, 71, 390, 744, 4, 3742, 32, 2], [0, 130078, 1632, 2837, 86918, 23, 7401, 569, 92, 538, 101087, 4, 28279, 4, 435, 34414, 136, 44603, 45, 128713, 13, 32, 2, 2, 130078, 67155, 2806, 188545, 23, 2

In [17]:
set(dataset["train"].features)

{'id', 'is_duplicate', 'qid1', 'qid2', 'question1', 'question2'}

In [18]:
pre_tokenizer_columns = set(dataset["train"].features)
encoded_dataset = dataset.map(preprocess_function, batched=True)
tokenizer_columns = list(set(encoded_dataset["train"].features) - pre_tokenizer_columns)
print("Columns added by tokenizer:", tokenizer_columns)

  0%|          | 0/40 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

Columns added by tokenizer: ['input_ids', 'attention_mask']


In [19]:
encoded_dataset["train"].features["is_duplicate"]

Value(dtype='int64', id=None)

In [21]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = encoded_dataset["train"].to_tf_dataset(
    columns=tokenizer_columns,
    label_cols=["is_duplicate"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_dataset = encoded_dataset["validation"].to_tf_dataset(
    columns=tokenizer_columns,
    label_cols=["is_duplicate"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [22]:
batch_size = 16

In [23]:
from transformers import create_optimizer

num_epochs = 3
batches_per_epoch = len(encoded_dataset["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(
    init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps
)
model.compile(optimizer=optimizer, loss=loss)

In [24]:
metric = load_metric("glue", "mrpc")

In [25]:
from transformers.keras_callbacks import KerasMetricCallback

metric_name = "accuracy"

def compute_metrics(eval_predictions):
    predictions, labels = eval_predictions
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

metric_callback = KerasMetricCallback(
    metric_fn=compute_metrics, eval_dataset=tf_validation_dataset)

In [ ]:
from tensorflow.keras.callbacks import TensorBoard

model_name = "xlm-roberta-base"

tensorboard_callback = TensorBoard(log_dir="./text_classification_model_save/logs")

callbacks = [metric_callback, tensorboard_callback]

model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=num_epochs,
    callbacks=callbacks,
)

Epoch 1/3
2500/2500 [==============================] - 865s 339ms/step - loss: 0.1460 - val_loss: 0.1155 - accuracy: 0.6156 - f1: 0.0000e+00
Epoch 2/3
1882/2500 [=====================>........] - ETA: 3:18 - loss: 0.0976

# References

## Notebooks
[Text Classification + TensorFlow](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/text_classification-tf.ipynb)

[Sentence Pair Classification + ALBERT + PyTorch](https://colab.research.google.com/github/NadirEM/nlp-notebooks/blob/master/Fine_tune_ALBERT_sentence_pair_classification.ipynb)

## Model

[xlm-roberta-base](https://huggingface.co/xlm-roberta-base)


## Dataset & Metric
[glue](https://huggingface.co/datasets/glue)